![](../images/rivacon_frontmark_combined_header.png)

# Interest Rate Swap 

In [1]:
import datetime as dt
from rivapy.marketdata.curves import DiscountCurve
from rivapy.instruments import IrFixedLegSpecification, IrFloatLegSpecification, InterestRateSwapSpecification, \
InterestRateBasisSwapSpecification
from rivapy.pricing import price
from rivapy.pricing.pricing_data import InterestRateSwapPricingData, InterestRateSwapLegPricingData, \
InterestRateSwapFloatLegPricingData
import math
%matplotlib inline

/home/thomas/.conda/envs/rivapy/lib/python3.8/site-packages/pyvacon/__init__.py:12: UserWarning: Imported pyvacon is just beta version.
  _warnings.warn('Imported pyvacon is just beta version.')
/home/thomas/RiVaPy/rivapy/__init__.py:9: UserWarning: Imported pyvacon is just beta version.
  warnings.warn('Imported pyvacon is just beta version.')


## Definition of an Interest Rate Swap

The most common type of an interest rate swap is the so called "plain vanilla" interest rate swap. Here, a company agrees to pay cashflows equal to a predetermined fixed interest rate on a notional principal during a predetermined number of years; in return, it receives interest payments on the same notional principal at a floating rate during the same number of years.


## Valuation of Interest Rate Swaps

The value of a swap can, for example, be regarded as the difference between two bonds. From the point of view of a fixed-rate payer, the value of the swap can be regarded as a long position in a floating-rate bond and a short position in a fixed-rate bond.

$$V_{swap} = B_{float} - B_{fix}$$

Alternatively, a plain vanilla interest rate swap can also be valued making the assumption that forward interest rates are realized. Here, cashflows of the floating leg are calculated assuming that the floating interest rate equals the forward interest rates. The value of the interest rate swap is then the sum of the present value of the net cashflows. This method will be applied in the following.

For a detailed discussion please refer to Hull, *Options, futures, and other derivatives, 8th Edition,* 2012, p. 148 ff.

In the following, we will use a slighly modified example taken from Hull, p. 161 to illustrate how a interest rate swap can be valued.

**Example:**

*Suppose that a financial institution has agreed to pay 6-month EURIBOR and receive 8% per annum (with semiannual compounding) on a notional principal of €100 million. The swap has a remaining life of 1.5 years. The EURIBOR rates with continuous compunding for 6-month, 12-month, and 18-month maturities are 10%, 10.5%, and 11%, respectively.*

### Valuation in terms of bond prices

**Fixed Leg**

The fixed-rate bond has three payment dates in 6 months, 12 months, and 18 months with a cashflow of €4 million at the first two payment dates and a cashflow of €4 million plus the notional of €100 million at the last payment date. These cashflows have to be discounted using the given EURIBOR rates and subsequently summed up to receive the present value of the fixed leg.

**Floating Leg**

The floating leg bond is worth the notional immediately after an interest payment because at this time, the bond is a "fair deal" where the borower pays EURIBOR for earch subsequent accrual period. Consequently, immediately before a payment, the bond is worth the notional plus the accrued interest. Also, this cashflow has to be discounted using the given EURIBOR rate.

In [2]:
#Fixed Leg
time_to_maturity = [0.5, 1.0, 1.5]
cf_fix = [4, 4, 104]
libor_rates = [0.1, 0.105, 0.11]

df = [math.exp(-lr * tm) for lr, tm in zip(libor_rates, time_to_maturity)]

pv_cf_fix = [cf * d for cf, d in zip(cf_fix, df)]
    
B_fix = sum(pv_cf_fix)
print(f'Fixed Leg Value = {B_fix}')


# Floating Leg
floating_rate = 2 * (math.exp(libor_rates[0] / 2) - 1)  # libor rate has to be converted from contiuous compounding to semiannual comp.
cf_float = 100 + (100 * floating_rate * 0.5) # 1/2 of converted libor_rate as semiannual payment
pv_cf_float = cf_float * df[0] # discounted with 3-month libor rate
print(f'Floating Leg Value = {pv_cf_float}')


# Interest Rate Swap Valuation (from a fixed-rate payer's perspective)
value = (pv_cf_float - B_fix)
print(f'Interest Rate Swap Value = {value}')

Fixed Leg Value = 95.58716101349117
Floating Leg Value = 100.00000000000001
Interest Rate Swap Value = 4.412838986508845



### Valuation in terms of forward rate agreements

As before, the value of the interest rate swap is given as the difference between the present value of the fixed and the floating leg.

**Fixed Leg**

The present value of the fixed leg is given by:

$$PV_{fixed}(t) = rN\sum_{i=1}^n\tau_iD_i$$

with $t$ as the valuation date, $r$ as the interest rate of the fixed leg, $N$ as the notional principal, $i$ as the $i^{th}$ cashflow, $\tau_i$ as the accrual period for the $i^{th}$ cashflow and $D_i$ as the discount factor of the $i^{th}$ period.




**Floating Leg**

The present value of the float leg is given by:

$$PV_{floating}(t)=N\sum_{i=1}^n(F_i+s)\tau_iD_i$$

with $F_i$ as the simply compounded forward rate defined as $\frac{1}{\tau_i}(\frac{D_{i-1}}{D_i}-1)$, $s$ as the floating spread and the rest defined as before.

Note that the discount factors used for the forward rate can be different from the discount factor used to discount the cashflows. Also, a different discount factors from the fixed leg are theoretically possible.

The following code manually calculates the value of the interest-rate swap. Thereby, the formula for the floating leg assuming that there is no spread $s$ is simplified as follows:

\begin{align}
PV_{floating}(t) & =N\sum_{i=1}^n\frac{1}{\tau_i}(\frac{D_{i-1}}{D_i}-1)\tau_iD_i \\
& =N\sum_{i=1}^n(\frac{D_{i-1}}{D_i}-1)D_i \\
& =N\sum_{i=1}^n(D_{i-1}-{D_i}) \\
& =N\cdot[(D_0-D_1)+(D_1-D_2)+ ... +(D_{n-2}-D_{n-1})+(D_{n-1}-D_n)] \\
& =N\cdot(D_0-D_n)
\end{align}

In [3]:
ttm = [0.5, 1, 1.5]
rates = [0.1, 0.105, 0.11]
yf = [0.25, 0.75, 1.25]
r = 0.08
N = 100

df = [math.exp(-r * tm) for r, tm in zip(rates, ttm)]
    
PV_fix = r * N * ((ttm[0] - 0) * df[0] + (ttm[1] - ttm[0]) * df[1] + (ttm[2] - ttm[1]) * df[2])
print('PV fixed leg =',PV_fix)

PV_fl = N * (1 - df[2])
print(f'PV floating leg = {PV_fl}')

print(f'Interest rate swap value = {PV_fl-PV_fix}')

PV fixed leg = 10.797790604699582
PV floating leg = 15.210629591208413
Interest rate swap value = 4.412838986508831


The following code shows the manual calculation of the example as it is done by Hull:

In [4]:
ttm = [0.5, 1.0, 1.5]
rates = [0.1, 0.105, 0.11]
N = 100 # Notional
m = 2 # compounding frequency

refdate = dt.datetime(2017, 1, 1)

days_to_maturity = [180, 360, 540]
dates = [refdate + dt.timedelta(days=d) for d in days_to_maturity]

# discount factors from constant rate
df = [math.exp(-r * t) for r, t in zip(rates, ttm)]

# Fixed leg
fix_cf = [4, 4, 4]
pv_fix = [cf * d for cf, d in zip(fix_cf, df)]
print(f'Present value fixed leg = {sum(pv_fix)}')

# Floating leg
floatrates = [m * (math.exp(rates[0] / m) - 1)]

for i in range(1, len(rates)):
    contrate = ((rates[i] * ttm[i] - rates[i-1] * ttm[i-1]) / (ttm[i] - ttm[i-1])) 
    floatrates.append(m * (math.exp(contrate / m) - 1))

float_cf = [fr * N / m for fr in floatrates]
float_pv = [cf * d for cf, d in zip(float_cf, df)]
print(f'Present value floating leg = {sum(float_pv)}')

# Net Cash-Flow
net_CF = [fl - fx for fl, fx in zip(float_cf, fix_cf)]
pv_net_cf = [cf * d for cf, d in zip(net_CF, df)]
value = sum(pv_net_cf)
print(f'Interest rate swap value = {value}')

Present value fixed leg = 10.797790604699582
Present value floating leg = 15.210629591208427
Interest rate swap value = 4.412838986508847


### Valuation using the interest rate swap specification

The pyvacon interest rate swap specification uses the valuation in terms of forward rate agreements. In order to price an interest rate swap, we need to create the necessary market data and setup the specification and pricing data.

#### Create the necessary market data

As a first step, we need to create a discount curve containing the discount factors to derive the present values of both, the fixed and the floating leg. Here, we use this discount curve to derive the forward rate discount factors as well. Theoretically, we could also use different discount curves for the fixed and floating legs as well as for the forward rates.

In [5]:
# Discount curve - we use these discount factors to get the present values of both the fixed and floating leg as well as 

object_id = "TEST_DC" 
refdatedc = dt.datetime(2017, 1, 1)
days_to_maturity = [180, 360, 540]
dates = [refdatedc + dt.timedelta(days=d) for d in days_to_maturity]
# discount factors from constant rate
rates = [0.10, 0.105, 0.11]
df = [math.exp(-r * d / 360) for r, d in zip(rates, days_to_maturity)]
dc = DiscountCurve(id=object_id, refdate=refdatedc, dates=dates, df=df)

#### Setup the specification

Here, we need to provide the start-, end-, and pay-dates for the different accrual periods. The floating leg additionally needs a reset date vector. Moreover, we need to define a vector of notionals since if needed, each accrual period may have a different notional. However, if the vector if of length 1, the same notional is applied to all periods.

Having defined the relevant information, we are able to  create the fixed- and floating leg. The fixed leg needs to be provided with the fixed rate; the float leg requires information about an eventual spread. Afterwards, the interest rate swap specification can be set.

In [6]:
# Create the vectors defining the statdates, enddates, paydates and reset dates
days_to_maturity = [0, 180, 360, 540]
dates = [dt.datetime(2017, 1, 1) + dt.timedelta(d) for d in days_to_maturity]

startdates = dates[:-1]
enddates = dates[1:]
paydates = enddates
resetdates = startdates
refdate = dates[0]

notionals = [100.0]

fixedleg = IrFixedLegSpecification(0.08, notionals, startdates, enddates, paydates, 'EUR', 'Act360')

spread = 0.00

floatleg = IrFloatLegSpecification(notionals, resetdates, startdates, enddates, paydates, 'EUR', 'test_udl',
                                   'Act360', spread)

ir_swap = InterestRateSwapSpecification('TEST_SWAP', 'DBK', 'COLLATERALIZED', 'EUR', paydates[-1], fixedleg, floatleg)


#### Setup the pricing data

A product may be priced in two different ways: One may either fill the respective pricing data needed for a special pricer (which inherits from th BasePricingData) and use the respective price method where just the pricing data is given. Another possibility is to use the price-method where the storages are given. In this case, the pricer will fill the needed pricing data according to the underlying and other data as specified in the product specification.

**Here we show the approach explicitely setting the pricing data.**  

The pricing data needs the following information:
- A discount curve for the pay- and receive leg
- A discount curve for the floating leg (here the receiving leg)
- A pricer
- A pricing request
- A valuation date
- The created interest rate swap specification
- An eventual FX-rate for the pay- and receive leg

In [7]:
pay_leg_pricing_data = InterestRateSwapLegPricingData(
    spec=ir_swap.getPayLeg(),
    discount_curve=dc,
    fx_rate=1.0,
    weight=-1.0
)
rec_leg_pricing_data = InterestRateSwapFloatLegPricingData(
    spec=ir_swap.getReceiveLeg(),
    discount_curve=dc,
    fixing_curve=dc,
    fx_rate=1.0,
    weight=1.0,
)
ir_swap_pricing_data = InterestRateSwapPricingData(
    spec=ir_swap,
    val_date=refdate,
    ccy='EUR',
    leg_pricing_data=[pay_leg_pricing_data, rec_leg_pricing_data],
    pricing_request=[]
)


#### Pricing

In [8]:
tic = dt.datetime.now()
pr = price(ir_swap_pricing_data)
print(f'runtime: {dt.datetime.now() - tic}')
pr.getPrice()

runtime: 0:00:00.000938


4.412838984470563

### Further Example
#### Create the necessary market data

This time, we distinguish between the discount curve used to discount the cashflows to present values and a discount curve for the calculation of the forward interest rates which we use as fixing curve.

In [9]:
refdatedc = dt.datetime(2017, 1, 1)
days_to_maturity = [1, 180, 365, 720, 3 * 365, 4 * 365, 10 * 365]
dates = [refdatedc + dt.timedelta(days=d) for d in days_to_maturity]

# Discount curve
object_id = "TEST_DC" 
dsc_rate = 0.01
df = [math.exp(-d / 365.0 * dsc_rate) for d in days_to_maturity]
dc = DiscountCurve(id=object_id, refdate=refdatedc, dates=dates, df=df)

# Fixing curves
object_id = "TEST_fwd_"
fwd_rate = {'3m': 0.05, '6m': 0.052}
fwd_df = {t: [math.exp(-d / 365.0 * r) for d in days_to_maturity] for t, r in fwd_rate.items()}
fwd_dc = {t: DiscountCurve(id=object_id + t, refdate=refdatedc, dates=dates, df=df) for t, df in fwd_df.items()}


#mkt_plot.curve(dc,range(1,10*365), refdatedc)
#mkt_plot.curve(fwd_dc,range(1,10*365), refdatedc)

#### Setup the specification
In our example we are using one fixed leg and one floating leg (with just two payment and fixing dates)
The swap needs to get vectors with start, end, and payment dates (floater additionally need a rest date vector). The i-th accrual period is defined by the i-th entry of start and end vectors.
Note that this kind of swap has only floating legs with floating rate period equal to accrual period.

In [10]:
# Create the vectors defining the statdates, enddates, paydates and reset dates
days_to_maturity = [0, 180, 360, 540]
dates = [dt.datetime(2017,1,1) + dt.timedelta(days=d) for d in days_to_maturity]

startdates = dates[:-1]
enddates = dates[1:]
paydates = enddates
resetdates = startdates
refdate = dates[0]

# We need a vector of notionals since if needed, each accrual period may have a different notional
# However, if the vector is of length 1 the same notional is applied to all periods

notionals = [100.0]

fixedleg = IrFixedLegSpecification(0.03, notionals, startdates, enddates, paydates,'EUR', 'Act365Fixed')
spread = 0.00
floatleg = IrFloatLegSpecification(notionals, resetdates, startdates, enddates, paydates,'EUR', 'test_udl',
                                   'Act365Fixed', spread)

ir_swap = InterestRateSwapSpecification('TEST_SWAP', 'DBK', 'COLLATERALIZED', 'EUR', paydates[-1], fixedleg, floatleg)

#### Setup the pricing data

In [11]:
pay_leg_pricing_data = InterestRateSwapLegPricingData(
    spec=ir_swap.getPayLeg(),
    discount_curve=dc,
    fx_rate=1.0,
    weight=-1.0
)
rec_leg_pricing_data = InterestRateSwapFloatLegPricingData(
    spec=ir_swap.getReceiveLeg(),
    discount_curve=dc,
    fixing_curve=fwd_dc['6m'],
    fx_rate=1.0,
    weight=1.0,
)
ir_swap_pricing_data = InterestRateSwapPricingData(
    spec=ir_swap,
    val_date=refdate,
    ccy='EUR',
    leg_pricing_data=[pay_leg_pricing_data, rec_leg_pricing_data],
    pricing_request=[]
)


#### Pricing

In [12]:
tic = dt.datetime.now()
pr = price(ir_swap_pricing_data)
print(f'runtime: {dt.datetime.now() - tic}')
pr.getPrice()

runtime: 0:00:00.013420


3.3213902674791154

## Tenor Basis Swap

There are two types of tenor basis swaps:
- In most currencies, a tenor basis swap consists of two float legs (e.g. 3M-USD-Libor vs 6M-USD-Libor) where the on the leg with the shorter tenor (in this case the 3M leg) there is an additional spread (the "basis").
- In EUR, a the "basis" is quoted as the difference of swap rates between two vanilla swaps with different float leg tenor (e.g. [swap rate of 6M-Euribor-Swap] - [swap rate of 3M-Euribor-Swap]). This is equivalent to a swap with three legs:
    - a floating leg which pays 3M-Euribor
    - a floating leg which receives 6M-Euribor
    - a fixed leg which pays the basis (usually annually)

In either case, if the discount curve and one of the forward (projection) curve are known, a strip of tenor basis swap quotes can be used to bootstrap the other forward curve.

In [13]:
# Set up curves
# Discount curve
object_id = "TEST_DC"
dsc_rate = 0.01
df = [math.exp(-d / 365.0 * dsc_rate) for d in days_to_maturity]
dc = DiscountCurve(id=object_id, refdate=refdatedc, dates=dates, df=df)

# Fixing curves
object_id = "TEST_fwd_"
fwd_rate = {'3m': 0.05, '6m': 0.052}
fwd_df = {t: [math.exp(-d / 365.0 * r) for d in days_to_maturity] for t, r in fwd_rate.items()}
fwd_dc = {t: DiscountCurve(id=object_id + t, refdate=refdatedc, dates=dates, df=df) for t, df in fwd_df.items()}

In [14]:
# set up two payer swaps (receive 3m/6m-Euribor, pay fixed)
refdate = dt.datetime(2017, 1, 1)
days = 720
tenor_in_days = {'3m': 90, '6m': 180, '12m': 360}
spread = 0.002
swap_rate_3m = .03
swap_rate_6m = swap_rate_3m + spread
swap_rates = {'3m': swap_rate_3m, '6m': swap_rate_6m}
days_to_maturity = {t: list(range(0, days + 1, tenor_days)) for t, tenor_days in tenor_in_days.items()}
notionals = [100.0]
ccy = 'EUR'
swaps = {}
swap_pricing_data = {}
t_fix = '12m'
dates_fix = [refdate + dt.timedelta(days=d) for d in days_to_maturity[t_fix]]

for t_fl in ['3m', '6m']:
# set up specification
    tenor_days_fl = tenor_in_days[t_fl]
    dates_fl = [refdate + dt.timedelta(days=d) for d in days_to_maturity[t_fl]]

    fixedleg = IrFixedLegSpecification(swap_rates[t_fl], notionals, dates_fix[:-1], dates_fix[1:], dates_fix[1:], ccy, 'Act365Fixed')
    floatleg = IrFloatLegSpecification(notionals, dates_fl[:-1], dates_fl[:-1], dates_fl[1:], dates_fl[1:], ccy, 'test_udl', 'Act365Fixed', 0.)
    swaps[t_fl] = InterestRateSwapSpecification('TEST_SWAP', 'DBK', 'COLLATERALIZED', ccy, dates_fl[-1], fixedleg, floatleg)
#set up pricing data

    pay_leg_pricing_data = InterestRateSwapLegPricingData(
        spec=swaps[t_fl].getPayLeg(),
        discount_curve=dc,
        fx_rate=1.0,
        weight=-1.0
    )
    rec_leg_pricing_data = InterestRateSwapFloatLegPricingData(
        spec=swaps[t_fl].getReceiveLeg(),
        discount_curve=dc,
        fixing_curve=fwd_dc[t_fl],
        fx_rate=1.0,
        weight=1.0,
    )
    swap_pricing_data[t_fl] = InterestRateSwapPricingData(
        spec=ir_swap,
        val_date=refdate,
        ccy='EUR',
        leg_pricing_data=[pay_leg_pricing_data, rec_leg_pricing_data],
        pricing_request=[]
    )



In [15]:
# Price the swaps
price_3m = price(swap_pricing_data['3m']).getPrice()
price_6m = price(swap_pricing_data['6m']).getPrice()
price_diff = price_6m - price_3m
print(f"3m payer swap: {price_3m}")
print(f"6m payer swap: {price_6m}")
print(f"Difference: {price_diff}")      


3m payer swap: 3.9837402680778116
6m payer swap: 4.043327258163279
Difference: 0.059586990085467306


---

In [16]:
# Set up basis swap

float_leg_3m = swaps['3m'].getReceiveLeg()
float_leg_6m = swaps['6m'].getReceiveLeg()
spread_leg = IrFixedLegSpecification(spread, notionals, dates_fix[:-1], dates_fix[1:], dates_fix[1:], ccy,
                                   'Act365Fixed')

basis_swap = InterestRateBasisSwapSpecification('xx', 'dummy_issuer', 'COLLATERALIZED', ccy, dates_fix[-1], float_leg_6m, float_leg_3m, spread_leg)

float_leg_3m_pricing_data = InterestRateSwapFloatLegPricingData(
    spec=float_leg_3m,
    discount_curve=dc,
    fixing_curve=fwd_dc['3m'],
    fx_rate=1.0,
    weight=-1.0)

float_leg_6m_pricing_data = InterestRateSwapFloatLegPricingData(
    spec=float_leg_6m,
    discount_curve=dc,
    fixing_curve=fwd_dc['6m'],
    fx_rate=1.0,
    weight=1.0)

spread_leg_pricing_data = InterestRateSwapLegPricingData(
    spec=spread_leg,
    discount_curve=dc,
    fx_rate=1.0,
    weight=-1.0)

basis_swap_pricing_data = InterestRateSwapPricingData(
    spec=basis_swap,
    val_date=refdate,
    ccy=ccy,
    leg_pricing_data=[float_leg_6m_pricing_data, float_leg_3m_pricing_data, spread_leg_pricing_data],
    pricing_request=[]
)


In [17]:
price_bs = price(basis_swap_pricing_data).getPrice()
print(f"3m/6m basis swap: {price_bs}")

3m/6m basis swap: 0.05958699008546775
